# Bidirectional Vanilla RNN corpus Otomi

In [1]:
from reccurrent_model import biRNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np
import torch

In [2]:
file1 = load(open('pickle_objects/predata','rb'))
print(file1[0], len(file1))

[[['n', 'v', 'B-psd'], ['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['μ', 'v', 'I-stem'], ['d', 'v', 'I-stem'], ['i', 'v', 'I-stem']], [['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['ε', 'v', 'I-stem'], ['p', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['í', 'v', 'I-stem']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['t', 'v', 'B-lig'], ["'", 'v', 'B-stem'], ['μ', 'v', 'I-stem'], ['n', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-stem'], ['ä', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['r', 'v', 'I-stem'], ['a', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['á', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['k', 'obl', 'B-stem'], ['o', 'obl', 'I-stem']], [['c', 'obl', 'B-stem'], ['h', 'obl', 'I-stem'], ['í', 'obl', 'I-stem'], ['k', 'obl', 'I-stem'], ['ó', 'obl', 'I-stem'], ['h', 'obl', 'I-stem'], ['t', 'obl', 'I-

In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_sents, train_tags = zip(*train_pairs)

1196 590


In [5]:
model = biRNN(train_sents, train_tags)

In [6]:
%%time
model.train_model(its=150)

100%|██████████| 150/150 [25:30<00:00, 10.20s/it]

CPU times: user 4h 36min 45s, sys: 42.5 s, total: 4h 37min 28s
Wall time: 25min 30s


### Evaluation

In [7]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = model.forward(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [8]:
y_true = list(chain(*y_true))
y_pred = list(chain(*y_pred))

In [9]:
labels = list(set(y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(y_true, y_pred))

Accuracy: 0.7648679103543258


In [10]:
prec, rec, f1, supp = precision_recall_fscore_support(y_true, y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
B-y,0.0,0.0,0.0,4.0
I-3.imp,0.0,0.0,0.0,7.0
I-1.irr,0.0,0.0,0.0,2.0
I-loco,0.0,0.0,0.0,3.0
B-3.pss,0.0,0.0,0.0,8.0
...,...,...,...,...
B-dónde,0.0,0.0,0.0,1.0
I-y,0.0,0.0,0.0,1.0
B-tal.vez,0.0,0.0,0.0,1.0
B-solo,0.0,0.0,0.0,2.0


In [12]:
print(results.to_string())

               Precision    Recall        F1  Support
B-y             0.000000  0.000000  0.000000      4.0
I-3.imp         0.000000  0.000000  0.000000      7.0
I-1.irr         0.000000  0.000000  0.000000      2.0
I-loco          0.000000  0.000000  0.000000      3.0
B-3.pss         0.000000  0.000000  0.000000      8.0
B-que           0.000000  0.000000  0.000000      4.0
I-1.enf         0.000000  0.000000  0.000000      6.0
I-dual.exc      0.000000  0.000000  0.000000     22.0
B-1.icp         0.614865  1.000000  0.761506     91.0
B-encl          0.000000  0.000000  0.000000      1.0
B-pl.exc        0.000000  0.000000  0.000000     72.0
B-p.loc         0.000000  0.000000  0.000000      4.0
I-que           0.000000  0.000000  0.000000      4.0
B-comp          0.000000  0.000000  0.000000      1.0
B-mucho         0.000000  0.000000  0.000000      1.0
B-2.cnt         0.000000  0.000000  0.000000      9.0
B-com           0.000000  0.000000  0.000000      1.0
I-3.pot         0.882353  0.

In [13]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.5384877261394624
Average recall: 0.7273914936855858
Average F1: 0.6181199291127571
